In [408]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium as folium

In [409]:
df = pd.read_csv('map_data_extended.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41,880,129.0,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,342200,NEAR BAY


In [410]:
# Create a copy of the dataframe
map_data = df.copy()

In [411]:
map_data.sample(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
5834,-118.33,34.20,43,1325,254.0,613,248,3.6071,289000,<1H OCEAN
2775,-114.73,33.43,24,796,243.0,227,139,0.8964,59200,INLAND


In [412]:
# Count unique values in ocean_proximity
map_data['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [413]:
# Find the highest and lowest values in the latitude column
max_lat = map_data['latitude'].max()
min_lat = map_data['latitude'].min()

# Find the highest and lowest values in the longitude column
max_long = map_data['longitude'].max()
min_long = map_data['longitude'].min()

In [414]:
# Use the max and min values to create a map of the data
map = folium.Map(location=[(max_lat + min_lat) / 2, (max_long + min_long) / 2], zoom_start=5)

In [415]:
h_ocean = map_data.loc[map_data['ocean_proximity'] == '<1H OCEAN'].sample(3)
inland = map_data.loc[map_data['ocean_proximity'] == 'INLAND'].sample(3)
near_ocean = map_data.loc[map_data['ocean_proximity'] == 'NEAR OCEAN'].sample(3)
near_bay = map_data.loc[map_data['ocean_proximity'] == 'NEAR BAY'].sample(3)
island = map_data.loc[map_data['ocean_proximity'] == 'ISLAND'].sample(3)

In [416]:
# Add all the dataframe above together
merged_df = pd.concat([h_ocean, inland, near_ocean, near_bay, island], ignore_index=True)

In [417]:
# # Add the markers to the map
# for lat, long, proximity in zip(merged_df['latitude'], merged_df['longitude'], merged_df['ocean_proximity']):
#     folium.Marker([lat, long], popup=proximity).add_to(map) 


In [418]:
# Fix colors for the different ocean proximity
def select_color(row):
    if row['ocean_proximity'] == '<1H OCEAN':
        return 'orange'
    elif row['ocean_proximity'] == 'INLAND':
        return 'blue'
    elif row['ocean_proximity'] == 'NEAR OCEAN':
        return 'red'
    elif row['ocean_proximity'] == 'NEAR BAY':
        return 'green'
    return 'pink'


In [419]:
merged_df['color'] = merged_df.apply(select_color, axis=1)

In [420]:
for _, house in merged_df.iterrows():
    folium.Marker(
        location=[house['latitude'], house['longitude']],
        popup=house['median_income'],
        tooltip=house['households'],
        # icon=folium.Icon(color=house['color']),
        icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: blue">{house['median_house_value']}</div>"""),
        marker_icon='cloud'
    ).add_to(map);

In [421]:
# Save the map as html
map.save('map.html')

In [422]:
# Make an empty map
map2 = folium.Map(location=[(max_lat + min_lat) / 2, (max_long + min_long) / 2], zoom_start=7)

# add marker one by one on the map
for i in range(0,len(merged_df)):
    html=f"""
        <div style="font-family:serif; padding:0px;color:{merged_df.iloc[i]['color']}">
        <h5 style="text-align:center">{merged_df.iloc[i]['ocean_proximity']}</h5>
        <p>Here are the details of the house:</p>
        <ul>
            <li>Median Income: ${merged_df.iloc[i]['median_income']}</li>
            <li>Median House Value: ${merged_df.iloc[i]['median_house_value']}</li>
        </ul>
        </p>
        </div>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[merged_df.iloc[i]['latitude'], merged_df.iloc[i]['longitude']],
        popup=popup,
        tooltip=merged_df.iloc[i]['ocean_proximity'],
        icon=folium.DivIcon(html=f"""
            <div>
			<svg version="1.1" id="Capa_1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="50" height="50" x="100px" y="100px" fill={merged_df.iloc[i]['color']}
	 		viewBox="0 0 415.482 415.482" style="enable-background:new 0 0 415.482 415.482;" xml:space="preserve">
			<g>
			<path d="M92.15,326.436c2.425-1.113,4.775-2.488,6.99-4.12c3.794-2.794,6.953-6.919,10.297-11.287
			c0.752-0.982,1.503-1.963,2.262-2.931c1.584-2.02,3.954-3.267,6.515-3.43c0.19-0.012,0.379-0.018,0.568-0.018
			c2.361,0,4.637,0.93,6.328,2.6c0.957,0.947,1.907,1.906,2.858,2.868c4.877,4.929,9.484,9.585,14.924,12.108
			c4.751,2.206,10.081,3.42,15.005,3.42c2.636,0,5.086-0.347,7.282-1.03c0.053-0.017,0.106-0.032,0.16-0.048
			c5.067-1.477,8.624-3.115,11.532-5.316c3.967-3.002,7.163-6.596,9.5-10.68l7.33-12.81l1.852-3.235
			c1.595-2.787,4.553-4.513,7.764-4.529c0.016,0,0.031,0,0.047,0c3.192,0,6.148,1.692,7.764,4.448l2.349,4.013
			c0.915,1.562,1.812,3.164,2.679,4.713c3.162,5.649,6.431,11.49,10.574,15.242c0.022,0.021,0.045,0.041,0.068,0.063
			c5.872,5.429,13.929,8.801,21.027,8.801c0.146,0,0.293-0.002,0.455-0.004c7.809-0.138,14.41-2.129,20.181-6.088
			c3.539-2.427,6.793-5.909,10.239-9.596c1.079-1.155,2.195-2.35,3.328-3.518c1.699-1.751,4.032-2.732,6.459-2.732
			c0.159,0,0.319,0.004,0.479,0.013c2.597,0.138,5.007,1.393,6.61,3.44c0.876,1.121,1.741,2.259,2.607,3.4
			c2.035,2.681,3.96,5.216,6.011,7.457c4.327,4.324,8.577,7.21,13.364,9.078c2.345,0.916,4.937,1.543,7.686,1.978
			c6.107,0.966,10.3,0.074,12.786-0.462c4.745-1.025,9.263-2.965,13.065-5.609c4.001-2.783,7.44-6.548,11.079-10.533l0.54-0.591
			c1.711-1.872,4.125-2.928,6.642-2.928c0.222,0,0.445,0.009,0.668,0.024c2.753,0.205,5.26,1.662,6.8,3.953l0.73,1.088
			c1.745,2.603,3.425,5.1,5.232,7.357l32.697-95.468H0L92.15,326.436z M93.241,246.337h20.5v9.75h-20.5V246.337z M80.741,246.337
			v9.75h-20.5v-9.75H80.741z"/>
			<path d="M387.482,162.047c-2.775,0-5.697,0-8.741,0v-16.762v-10.948v-30.544c-24.919,0-53.486,0-80.245,0v-22.5h31.5
			c2.624,0,4.75-2.126,4.75-4.75c0-2.623-2.126-4.75-4.75-4.75h-31.5v-0.666c0-4.419-3.582-8-8-8c-4.418,0-8,3.581-8,8v0.666h-31.5
			c-2.624,0-4.75,2.127-4.75,4.75c0,2.624,2.126,4.75,4.75,4.75h31.5v22.5c-43.984,0-79.5,0-79.5,0l-24.661,30.544
			c-9.681,0-15.339,0-15.339,0l-25.044,27.71c-12.825,0-20.452,0-20.452,0L80,207.539h335.482L387.482,162.047z M155.496,190.416h-30
			V179.17h30V190.416z M174.741,149.459h30v11.246h-30V149.459z M205.496,190.416h-30V179.17h30V190.416z M216.741,118.916h30v11.246
			h-30V118.916z M224.741,149.459h30v11.246h-30V149.459z M255.496,190.416h-30V179.17h30V190.416z M266.741,118.916h30v11.246h-30
			V118.916z M274.741,149.459h30v11.246h-30V149.459z M305.496,190.416h-30V179.17h30V190.416z M316.741,118.916h30v11.246h-30
			V118.916z M324.741,149.459h30v11.246h-30V149.459z M355.496,190.416h-30V179.17h30V190.416z"/>
			<path d="M378.785,328.756c-3.681-3.795-6.492-8.141-9.43-12.51c-4.125,4.514-8.075,8.93-13.123,12.441
			c-4.884,3.397-10.497,5.764-16.304,7.018c-3.589,0.773-8.737,1.504-15.968,0.572c-3.318-0.428-6.558-1.283-9.677-2.502
			c-6.449-2.516-11.74-6.357-16.594-11.236c-3.32-3.604-6.188-7.682-9.203-11.535c-4.737,4.883-9.309,10.408-14.936,14.268
			c-7.578,5.199-16.05,7.504-25.113,7.664c-0.204,0.004-0.409,0.006-0.613,0.006c-9.876,0-19.999-4.594-27.137-11.193
			c-6.524-5.908-10.577-14.566-14.977-22.076c-0.782-1.338-1.564-2.674-2.348-4.01c-3.061,5.348-6.121,10.693-9.181,16.043
			c-3.025,5.287-7.038,9.723-11.881,13.387c-4.376,3.313-9.235,5.264-14.446,6.781c-3.179,0.99-6.552,1.438-9.958,1.438
			c-6.516,0-13.154-1.639-18.795-4.256c-8.121-3.77-14.065-10.555-20.321-16.74c-4.393,5.602-8.506,11.641-14.305,15.912
			c-7.742,5.703-17.218,9.18-26.868,9.18c-1.12,0-2.241-0.047-3.364-0.143c-5.679-0.484-10.894-1.832-16.128-3.986
			c-4.681-1.926-8.636-5.029-12.565-8.141c-0.655-0.52-3.024-2.348-3.024-2.348s-3.75,4.299-5.658,6.049
			c-3.018,2.77-6.036,5.291-9.529,7.449c-2.852,1.697-5.764,3.121-8.873,4.293c-3.698,1.393-7.476,2.17-11.352,2.82v0.26
			c1.255,0.055,2.496,0.086,3.73,0.086c3.611,0,7.161-0.266,10.799-0.98c4.473-0.84,8.902-2.67,12.9-4.793
			c1.909-1.014,3.704-2.131,5.496-3.342c0.383-0.258,0.766-0.516,1.147-0.774c0.457-0.309,1.251-0.926,1.251-0.926
			s2.115,1.949,3.041,2.771c3.289,2.92,6.781,5.34,10.583,7.541c7.222,4.18,15.358,5.857,23.55,6.613
			c1.291,0.119,2.592,0.178,3.897,0.178c10.147,0,20.597-3.484,29.025-8.691c1.774-1.096,3.477-2.305,5.115-3.594
			c0.41-0.322,0.82-0.646,1.226-0.977c0.28-0.229,0.353-0.354,0.43-0.354c0.071,0,0.146,0.105,0.39,0.326
			c0.534,0.486,1.089,0.947,1.64,1.412c1.948,1.647,4.016,3.156,6.166,4.527c6.302,4.018,13.23,6.725,20.539,8.213
			c3.697,0.752,7.634,1.141,11.593,1.141c5.536,0,11.118-0.762,16.161-2.359c10.456-3.285,20.29-8.799,27.627-17.07
			c10.819,12.34,27.777,19.746,44.396,19.746c3.776,0,7.535-0.383,11.2-1.176c7.711-1.672,14.961-4.449,21.589-8.777
			c2.089-1.363,4.067-2.885,5.954-4.516c0.474-0.408,0.944-0.818,1.403-1.246c0.683,0.582,1.382,1.145,2.087,1.699
			c1.758,1.385,3.626,2.648,5.537,3.813c9.029,5.502,19.481,8.398,29.983,8.398c3.612,0,7.228-0.344,10.796-1.039
			c12.944-2.527,24.8-9.637,32.93-20.049c5.863,5.746,13.147,9.863,20.95,12.316c4.637,1.455,9.411,2.037,14.215,2.037
			c1.3,0,2.602-0.043,3.903-0.121v-0.303c-0.492-0.072-0.985-0.143-1.478-0.213C396.314,341.131,386.37,336.576,378.785,328.756z"/>
			</g>
			</svg>
            </div>""")
    ).add_to(map2)

In [423]:
map2.save('map2.html')